<a href="https://colab.research.google.com/github/kitomicms/python_chess_/blob/master/python_chess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [200]:
# python chess
# class Game():
#   def __init__(self):
#     board = Board()
#     self.board = board
#     self.turn = 'W'

# TDL
# [] starting position choice, newgame / by FEN


class Board():
  def __init__(self,fen=None):
    files = ['A','B','C','D','E','F','G','H']
    ranks = [1,2,3,4,5,6,7,8]
    board = []
    self.board = board
    counter = 1
    self.fen = fen
    
    # create all the squares
    for rank in reversed(ranks):
      for file in files:
        board.append(Square(rank,file,counter))
        counter += 1

    # add coordinates
    row_counter = 1
    for count,square in enumerate(board):
      
      if (count+1) % 8 == 0:
        x = 8
        y = row_counter
        row_counter +=1
      else:
        x = (count+1) % 8
        y = row_counter
      square.coordinate = (x,y)
    
    # # starting position:
    # if self.fen == None:
    #   # add pawns
    #   for square in board:
    #     if square.rank == 2:
    #       square.addPiece("P","W")
    #     elif square.rank == 7:
    #       square.addPiece("P","B")
    #   # add other pieces
    #   abc = ['R','N','B','Q','K','B','N','R']
    #   counter = 0
    #   for square in board:
    #     if square.rank == 1:
    #       square.addPiece(abc[counter],"W")
    #       counter += 1
    #   counter = 0
    #   for square in board:
    #     if square.rank == 8:
    #       square.addPiece(abc[counter],"B")
    #       counter += 1
    #   # set initial turn
    #   self.turn = 'W'
    # else:
    #   self.fen_to_board(fen)
    self.fen_to_board(fen)  
    
        
        

  def fen_to_board(self,fen):
    """
    Turn existing board to fen

    fen definition:
    1. board, seperate by /
    2. whose turn (w or b)
    3. castling (KQkq or -) shown letters only when available, if none then '-' is shown
    4. en passant square (e4 or -)
    5. halfmove clock : halfmove number since last capture or pawn advance
    6. fullmove number: start at 1 and increment after black move
    """
    p_dict = {'r':['R','B'],'n':['N','B'],'b':['B','B'],'q':['Q','B'],'k':['K','B'],'p':['P','B'],
              'R':['R','W'],'N':['N','W'],'B':['B','W'],'Q':['Q','W'],'K':['K','W'],'P':['P','W']}

    if not fen:
      fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
    
    breakdown = fen.split(" ")
    if len(breakdown) != 6:
      return
    positions = ''.join(breakdown[0].split("/"))
    turn = breakdown[1].upper()
    castling = breakdown[2] 
    enpassant = breakdown[3] 
    halfmove = breakdown[4]
    fullmove = breakdown[5]
    # manipulate positions string to 64 char
    positions_64 = ''
    for x in positions:
      if x in 'rnbqkpRNBQKP':
        positions_64 += x
      else:
        positions_64 = positions_64  + int(x) * '1'
    
    for count,square in enumerate(self.board):
      if positions_64[count].isnumeric():
        square.reset()
      else:
        keys = p_dict[positions_64[count]]
        square.addPiece(keys[0],keys[1]) #name,color

    self.turn = turn


  def to_fen(self):
    """
    Turn existing board to fen
    """
    positions_64 = ''
    p_dict = {'WR':'R','WN':'N','WB':'B','WQ':'Q','WK':'K','WP':'P',
              'BR':'r','BN':'n','BB':'b','BQ':'q','BK':'k','BP':'p',}
    for square in self.board:
      if square.piece_name in 'RNBQKP':
        positions_64 = positions_64 + p_dict[square.piece_color + square.piece_name]
      else:
        positions_64 = positions_64 + '1'
    newText = []
    
    step = 8
    textStr = [positions_64[i:i+step] for i in range(0,64,step)]
    for text in textStr:
      counter = 0
      newStr = ''
      for x in range(0,len(text)):
        if not text[x].isnumeric(): # alphapets
          newStr += text[x] 
        elif (text[x].isnumeric()) and  (len(text) - 1 == x): # last letter
          newStr += str(counter+1)
          counter = 0
        elif (text[x].isnumeric()) and text[x+1].isnumeric():
          counter += 1
        else:
          newStr += str(counter+1)
          counter = 0
      newText.append(newStr)
    newText = '/'.join(newText)
    return newText + ' ' + self.turn.lower() + ' - - 0 1'

  def current_turn(self):
    return self.turn

  def display_color(self):
    display_str = ''
    for count,square in enumerate(self.board):
      if (count+1) % 8 == 0:
          display_str = f"{display_str}{square.piece_color} "
          print(display_str)
          display_str = ''
      else:
        display_str = f"{display_str}{square.piece_color} "

  def display_matrix(self):
    display_str = ''
    for count,square in enumerate(self.board):
      if (count+1) % 8 == 0:
          display_str = f"{display_str}{square.coordinate} "
          print(display_str)
          display_str = ''
      else:
        display_str = f"{display_str}{square.coordinate} "

  def display_board(self):
      display_str = ''
      for count,square in enumerate(self.board):
        if (count+1) % 8 == 0:
          if square.piece_name in ['X',None]:
            display_str = f"{display_str}{square.coordinate[0]}{square.coordinate[1]} "
          else:
            display_str = f"{display_str}{square.piece_color}{square.piece_name} "
          print(display_str)
          display_str = ''
        else:
          if square.piece_name in ['X',None]:
            display_str = f"{display_str}{square.coordinate[0]}{square.coordinate[1]} "
          else:
            display_str = f"{display_str}{square.piece_color}{square.piece_name} "
      #print(display_str)
  
  def display_occupied(self):
    display_str = ''
    for count,square in enumerate(self.board):
      if (count+1) % 8 == 0:
          display_str = f"{display_str}{square.isOccupied} "
          print(display_str)
          display_str = ''
      else:
        display_str = f"{display_str}{square.isOccupied} "

  def square_info(self,rank,file):
    # what is on the square?
    
    for square in self.board:
      if square.rank == rank and square.file == file:
        return [square,square.piece_color,square.piece_name,square.coordinate]


  def makeMove(self,square1,square2,ignore_color=False):
    sq1_ = self.square_info(int(square1[1]),square1[0])[0]
    sq1_color = self.square_info(int(square1[1]),square1[0])[1]
    sq1_pieceName = self.square_info(int(square1[1]),square1[0])[2]
    sq2_ = self.square_info(int(square2[1]),square2[0])[0]
    sq2_color = self.square_info(int(square2[1]),square2[0])[1]
    sq2_pieceName = self.square_info(int(square2[1]),square2[0])[2]

    if self.turn != sq1_color and ignore_color == False:
      print("Not your turn")
      return 0

    if sq1_.isOccupied != False:
      if sq2_.isOccupied == False:
        sq1_.reset()
        sq2_.reset()
        sq2_.addPiece(sq1_pieceName,sq1_color)
        self.turn = 'W' if self.turn == 'B' else 'B'

      elif sq2_.isOccupied and sq1_color != sq2_color:
        sq1_.reset()
        sq2_.reset()
        sq2_.addPiece(sq1_pieceName,sq1_color)
        self.turn = 'W' if self.turn == 'B' else 'B'
      elif sq1_color == sq2_color:
        print("both square has same color piece")
      else:
        print("something else went wrong")
    else:
      print("Square 1 is Empty")
    # 1. MT
    # 2. Occupied,MT (King exception, path checked by opponent)
    # 3. Occupied,Occupied
    # 3.1 W,W
    # 3.2 W,B
  
  def availableMoves(self,square1):
    # according to current piece position and apply coordinate movement
    sq1_ = self.square_info(int(square1[1]),square1[0])[0]
    sq1_pieceColor = sq1_.piece_color
    sq1_x = sq1_.coordinate[0]
    sq1_y = sq1_.coordinate[1]
    
    # check if the square is occupied
    if sq1_.isOccupied:
      movesIndex = sq1_.piece.validMovesIndex
      possibleMoves = [(x+sq1_x,y+sq1_y) for (x,y) in movesIndex]
      impossibleMoves = []
      
      # find out moves that are not in the 8x8 grid
      for move in possibleMoves:
        if (move[0] <= 0) or (move[1] <= 0):
          impossibleMoves.append(move)
        elif (move[0] >= 9) or (move[1] >= 9):
          impossibleMoves.append(move)
      # remove all the impossible moves
      for move in impossibleMoves:
        possibleMoves.remove(move)
      impossibleMoves = []

      # check if dest position has own piece
      for move in possibleMoves:
        if self.squareByCoordinate(move).piece_color == sq1_pieceColor:
          impossibleMoves.append(move)
      # remove all the impossible moves
      for move in impossibleMoves:
        possibleMoves.remove(move)
      impossibleMoves = []
      # find out if other pieces is between current position and the dest position
      # Pawn
      ## 1. move forward
      ## 2. move diagonal to take opponent piece
      ## 3. en passant to take opponent piece, need to create a slot for available en passant square 

      # Knight
      ## no same color piece at dest position
      
      # King
      ## no same color piece at dest position
      
      # applied to Queen/Rook/Bishop
      if sq1_.piece_name in ['Q','R','B']:  
        for (x_,y_) in possibleMoves:
          x_distance = x_ - sq1_x
          y_distance = y_ - sq1_y

          if (abs(x_distance) == 1 or abs(x_distance) == 0) and \
          (abs(y_distance) == 1 or abs(y_distance) == 0): # all the 1 square moves
            pass
          elif (abs(x_distance) > 1 and abs(y_distance) == 0):
            # say rook move sideway
            if x_distance > 1:
              for x in range(1,x_distance):
                if self.squareByCoordinate((sq1_x+x,sq1_y)).isOccupied:
                  impossibleMoves.append((x_,y_))
                  break
            elif x_distance < 1:
              for x in range(1,-x_distance):
                if self.squareByCoordinate((-sq1_x+x,sq1_y)).isOccupied:
                  impossibleMoves.append((x_,y_))
                  break
          elif (abs(x_distance) == 0 and abs(y_distance) > 1):
            # say rook move up and down
            if y_distance > 1:
              for y in range(1,y_distance):
                if self.squareByCoordinate((sq1_x,y+sq1_y)).isOccupied:
                  impossibleMoves.append((x_,y_))
                  break
            elif y_distance < 1:
              for y in range(1,-y_distance):
                if self.squareByCoordinate((sq1_x,-y+sq1_y)).isOccupied:
                  impossibleMoves.append((x_,y_))
                  break
          elif abs(x_distance) == abs(y_distance):
            if x_distance > 1 and y_distance > 1:
              for x in range(1,x_distance):
                if self.squareByCoordinate((sq1_x+x,sq1_y+x)).isOccupied:
                  impossibleMoves.append((x_,y_))
                  break
            elif x_distance > 1 and y_distance < 1:
              for x in range(1,x_distance):
                if self.squareByCoordinate((sq1_x+x,sq1_y-x)).isOccupied:
                  impossibleMoves.append((x_,y_))
                  break
            elif x_distance < 1 and y_distance > 1:
              for x in range(1,-x_distance):
                if self.squareByCoordinate((sq1_x-x,sq1_y+x)).isOccupied:
                  impossibleMoves.append((x_,y_))
                  break
            elif x_distance < 1 and y_distance < 1:
              for x in range(1,-x_distance):
                if self.squareByCoordinate((sq1_x-x,sq1_y-x)).isOccupied:
                  impossibleMoves.append((x_,y_))
                  break
            else:
              print("Weird")  
          else: # should remain diagonal move
            print("Weird move")
        # remove all the impossible moves
        for move in impossibleMoves:
          possibleMoves.remove(move)
        impossibleMoves = []
      return possibleMoves # all the legit move based on piece's rules
    else:
      print("This square is not occupied by any pieces")

    # remove the invalid move due to blockage
    
  def squareByCoordinate(self,coordinate):
    # x files , y ranks
    files = ['A','B','C','D','E','F','G','H']
    ranks = [1,2,3,4,5,6,7,8]
    return self.square_info(ranks[-(coordinate[1])],files[coordinate[0]-1])[0]

  def coordinate_to_letter(self,coordinate):
    files = ['A','B','C','D','E','F','G','H']
    ranks = [1,2,3,4,5,6,7,8]
    return str(files[coordinate[0]-1]) + str(ranks[-(coordinate[1])])

  def undoMove(self,square1,square1_piece,square2,square2_piece):
    # piece should be like "WB" aka white bishop
    
    # need to handle empty cells

    square1.reset()
    #print(square1_piece)
    square1.addPiece(square1_piece[1],square1_piece[0]) # name, color

    #sq2_ = self.square_info(int(square2[1]),square2[0])[0]
    square2.reset()
    square2.addPiece(square2_piece[1],square2_piece[0])

    self.turn == 'B' if self.turn == 'W' else 'W'


  def legalMoves(self, square1): # E4, E5
    # import available moves
    possibleMoves = self.availableMoves(square1)
    #print(possibleMoves)
    impossibleMoves = []

    sq1_ = self.square_info(int(square1[1]),square1[0])[0]
    sq1_piece_name = sq1_.piece_name
    sq1_piece_color = sq1_.piece_color
    sq1_piece = str(sq1_.piece_color) + str(sq1_.piece_name)

    # check if being checked after move
    for coordinate in possibleMoves:
      
      square2 = self.coordinate_to_letter(coordinate) # square class
      sq2_ = self.squareByCoordinate(coordinate)

      # save for undo
      #sq2_piece_name = sq2_.piece_name
      #sq2_piece_color = sq2_.piece_color
      sq2_piece = str(sq2_.piece_color) + str(sq2_.piece_name)

      # make the pseudo move
      self.makeMove(square1,square2,ignore_color=True)


      if self.isChecked(sq1_piece_color):
        impossibleMoves.append(coordinate)
      self.undoMove(sq1_,sq1_piece,sq2_,sq2_piece)
    for move in impossibleMoves:
        possibleMoves.remove(move)
    return possibleMoves
    # king move
    ## still being checked?

    # other move
    ## allow to be checked?
    
  def isChecked(self,color): # 'W' or 'B'

    legalMoves = []

    enemy_color = 'B' if color == 'W' else 'W'  

    # define king position
    for square in self.board:
      if square.piece_name == "K" and square.piece_color == color:
        kingpos = square.coordinate
        break
        
    # find the edge of the board and loop through the file/rank to get the closest unit

    # loop up Bishop/Queen
    bishop = Bishop('B','W')
    movesIndex = bishop.validMovesIndex

    ## check out of grid and if has target piece checking the king
    for (x,y) in movesIndex:
      if (x+kingpos[0] <= 0) or (y+kingpos[1] <= 0):
        pass
      elif (x+kingpos[0] >= 9) or (y+kingpos[1] >= 9):
        pass
      else:
        sq_ = self.squareByCoordinate((x+kingpos[0],y+kingpos[1]))
        if sq_.piece_name in ['B','Q'] and sq_.piece_color == enemy_color :
          if not self.inBetween(kingpos,sq_.coordinate):
            #legalMoves.append(sq_.coordinate)
            return True

     # loop up rook/queen
    rook = Rook('R','W')
    movesIndex = rook.validMovesIndex

    ## check out of grid and if has target piece checking the king
    for (x,y) in movesIndex:
      if (x+kingpos[0] <= 0) or (y+kingpos[1] <= 0):
        pass
      elif (x+kingpos[0] >= 9) or (y+kingpos[1] >= 9):
        pass
      else:
        sq_ = self.squareByCoordinate((x+kingpos[0],y+kingpos[1]))
        if sq_.piece_name in ['R','Q'] and sq_.piece_color == enemy_color :
          if not self.inBetween(kingpos,sq_.coordinate):
            #legalMoves.append(sq_.coordinate)
            return True

 # loop up knight
    knight = Knight('N','W')
    movesIndex = rook.validMovesIndex

    ## check out of grid and if has target piece checking the king
    for (x,y) in movesIndex:
      if (x+kingpos[0] <= 0) or (y+kingpos[1] <= 0):
        pass
      elif (x+kingpos[0] >= 9) or (y+kingpos[1] >= 9):
        pass
      else:
        sq_ = self.squareByCoordinate((x+kingpos[0],y+kingpos[1]))
        if sq_.piece_name in ['N'] and sq_.piece_color == enemy_color :
          return True

 # loop up pawn
    if color == 'W':
      movesIndex = [(-1,-1),(1,-1)]
    else:
      movesIndex = [(1,1),(-1,1)]

    ## check out of grid and if has target piece checking the king
    for (x,y) in movesIndex:
      if (x+kingpos[0] <= 0) or (y+kingpos[1] <= 0):
        pass
      elif (x+kingpos[0] >= 9) or (y+kingpos[1] >= 9):
        pass
      else:
        sq_ = self.squareByCoordinate((x+kingpos[0],y+kingpos[1]))
        if sq_.piece_name in ['P'] and sq_.piece_color == enemy_color :
          return True



    

  def inBetween(self, sq1_coordinate, sq2_coordinate): 
    sq1_x = sq1_coordinate[0]
    sq1_y = sq1_coordinate[1]

    x_ = sq2_coordinate[0]
    y_ = sq2_coordinate[1]
    
    x_distance = x_ - sq1_x
    y_distance = y_ - sq1_y

    if (abs(x_distance) == 1 or abs(x_distance) == 0) and \
    (abs(y_distance) == 1 or abs(y_distance) == 0): # all the 1 square moves
      pass
    elif (abs(x_distance) > 1 and abs(y_distance) == 0):
      # say rook move sideway
      if x_distance > 1:
        for x in range(1,x_distance):
          if self.squareByCoordinate((sq1_x+x,sq1_y)).isOccupied:
            
            return True
      elif x_distance < 1:
        for x in range(1,-x_distance):
          if self.squareByCoordinate((-sq1_x+x,sq1_y)).isOccupied:
            return True
    elif (abs(x_distance) == 0 and abs(y_distance) > 1):
      # say rook move up and down
      if y_distance > 1:
        for y in range(1,y_distance):
          if self.squareByCoordinate((sq1_x,y+sq1_y)).isOccupied:
            return True
      elif y_distance < 1:
        for y in range(1,-y_distance):
          if self.squareByCoordinate((sq1_x,-y+sq1_y)).isOccupied:
            return True
    elif abs(x_distance) == abs(y_distance):
      if x_distance > 1 and y_distance > 1:
        for x in range(1,x_distance):
          if self.squareByCoordinate((sq1_x+x,sq1_y+x)).isOccupied:
            return True
      elif x_distance > 1 and y_distance < 1:
        for x in range(1,x_distance):
          if self.squareByCoordinate((sq1_x+x,sq1_y-x)).isOccupied:
            return True
      elif x_distance < 1 and y_distance > 1:
        for x in range(1,-x_distance):
          if self.squareByCoordinate((sq1_x-x,sq1_y+x)).isOccupied:
            return True
      elif x_distance < 1 and y_distance < 1:
        for x in range(1,-x_distance):
          if self.squareByCoordinate((sq1_x-x,sq1_y-x)).isOccupied:
            return True
      else:
        print("Weird")  
    else: # should remain diagonal move
      print("Weird move")

  def isCheckmated(self,color):
    # in check
    # and no legal moves
    for square in board:
      if self.legalMoves(square):
        return True
    

  def isStalemate(side):

    # no valid moves
    # every move is being checked
    if not isChecked(side):
      print("not in check")

    if len(turn.legal_moves()) == 0:
      print("no legal moves")

    return True
  def castling():
    pass
  def enpassant():
    pass
    # need to check if oppenent pawn made 2 step move at first and never move after that

  def fen():
    # read fen
    pass


# row = rank, col = file
class Square():
  def __init__(self, rank,file,coordinate,piece=None):
        self.rank = rank
        self.file = file
        self.isOccupied = False
        self.piece = None
        self.piece_name = 'X'
        self.piece_color = 'X'
        self.coordinate = coordinate

  def addPiece(self,name,color):
    if name == 'P':
      self.piece = Pawn(name,color)
      self.piece_name = name
      self.piece_color = color
      self.isOccupied = True
    elif name == 'K':
      self.piece = King(name,color)
      self.piece_name = name
      self.piece_color = color
      self.isOccupied = True
    elif name == 'Q':
      self.piece = Queen(name,color)
      self.piece_name = name
      self.piece_color = color
      self.isOccupied = True
    elif name == 'R':
      self.piece = Rook(name,color)
      self.piece_name = name
      self.piece_color = color
      self.isOccupied = True
    elif name == 'N':
      self.piece = Knight(name,color)
      self.piece_name = name
      self.piece_color = color
      self.isOccupied = True
    elif name == 'B':
      self.piece = Bishop(name,color)
      self.piece_name = name
      self.piece_color = color
      self.isOccupied = True
    elif name == 'X':
      self.reset()
    else:
      print("invalid piece name entered")
      

  def reset(self):
    self.isOccupied = False
    self.piece = None
    self.piece_name = 'X'
    self.piece_color = 'X'

class Piece():
  def __init__(self,name,color):
    self.name = name
    self.color = color
    

class King(Piece):
  def __init__(self, name,color):
    self.name = name
    self.color = color
    self.validMovesIndex = []
    self.validMovesIndex.append((1,0))
    self.validMovesIndex.append((-1,0))
    self.validMovesIndex.append((0,1))
    self.validMovesIndex.append((0,-1))
    self.validMovesIndex.append((1,1))
    self.validMovesIndex.append((1,-1))
    self.validMovesIndex.append((-1,1))
    self.validMovesIndex.append((-1,-1))
class Queen(Piece):
  # moves rule
  def __init__(self, name,color):
    self.name = name
    self.color = color
    self.validMovesIndex = []
    for x in [1,2,3,4,5,6,7]:
      self.validMovesIndex.append((x,0))
      self.validMovesIndex.append((-x,0))
      self.validMovesIndex.append((0,x))
      self.validMovesIndex.append((0,-x))
      self.validMovesIndex.append((x,x))
      self.validMovesIndex.append((x,-x))
      self.validMovesIndex.append((-x,x))
      self.validMovesIndex.append((-x,-x))
class Rook(Piece):
  def __init__(self, name,color):
    self.name = name
    self.color = color
    self.validMovesIndex = []
    for x in [1,2,3,4,5,6,7]:
      self.validMovesIndex.append((x,0))
      self.validMovesIndex.append((-x,0))
      self.validMovesIndex.append((0,x))
      self.validMovesIndex.append((0,-x))
      
class Pawn(Piece):
  def __init__(self, name,color):
    self.name = name
    self.color = color
    # depends on which color
    self.validMovesIndex = []
    if self.color == 'W':
      self.validMovesIndex.append((0,-1))
      # what if it is the first move
    else:
      self.validMovesIndex.append((0,1))
    self.movesHistory = []

class Knight(Piece):
  def __init__(self, name,color):
    self.name = name
    self.color = color
    self.validMovesIndex = []
    self.validMovesIndex.append((1,2))
    self.validMovesIndex.append((-1,2)) 
    self.validMovesIndex.append((1,-2)) 
    self.validMovesIndex.append((-1,-2)) 
    self.validMovesIndex.append((2,1)) 
    self.validMovesIndex.append((-2,1)) 
    self.validMovesIndex.append((2,-1))  
    self.validMovesIndex.append((-2,-1))  

class Bishop(Piece):
  def __init__(self, name,color):
    self.name = name
    self.color = color
    self.validMovesIndex = []
    for x in [1,2,3,4,5,6,7]:
      self.validMovesIndex.append((x,x))
      self.validMovesIndex.append((x,-x))
      self.validMovesIndex.append((-x,x))
      self.validMovesIndex.append((-x,-x))

def test_move(case,answer):
  case = sorted(case)
  answer = sorted(answer)
  
  if case == answer:
    
    print("Test passed")
  else:
    print(case,"is passed through")
    print(answer,"is expected")
    print("Test failed")


In [201]:
board = Board()
board.current_turn()
board.makeMove("H1","D4")
board.makeMove("H7","H6")
board.makeMove("D2","D3")
board.makeMove("B8","E4")
board.makeMove("H2","H3")
board.makeMove("E7","E5")
board.makeMove("B2","B3")
board.display_board()

# Test black rook
test_move(board.availableMoves("H8"),[(8,2)])

# Test white knight
test_move(board.availableMoves("G1"),[(6,6)])
# Test Black knight
test_move(board.availableMoves("E4"),[(4,3),(6,3),(3,4),(3,6),(4,7),(6,7),(7,4),(7,6)])

# Test white pawn
test_move(board.availableMoves("H3"),[(8,5)])

# Test white pawn
test_move(board.availableMoves("A8"),[(2,1)])

# Test white pawn
test_move(board.availableMoves("D3"),[])

# Test white bishop
test_move(board.availableMoves("C1"),[(1,6),(2,7),(4,7),(5,6),(6,5),(7,4),(8,3)])

# Test white bishop 2
test_move(board.availableMoves("F1"),[])

# Test white queen
test_move(board.availableMoves("D1"),[(4,7)])

# Test black queen
test_move(board.availableMoves("D8"),[(5,2),(6,3),(7,4),(8,5)])

# Test black bishop
test_move(board.availableMoves("F8"),[(5,2),(4,3),(3,4),(2,5),(1,6)])

# Test black bishop 2
test_move(board.availableMoves("C8"),[])

BR 21 BB BQ BK BB BN BR 
BP BP BP BP 52 BP BP 82 
13 23 33 43 53 63 73 BP 
14 24 34 44 BP 64 74 84 
15 25 35 WR BN 65 75 85 
16 WP 36 WP 56 66 76 WP 
WP 27 WP 47 WP WP WP 87 
WR WN WB WQ WK WB WN 88 
Test passed
Test passed
Test passed
Test passed
Test passed
Test passed
Test passed
Test passed
Test passed
Test passed
Test passed
Test passed


In [202]:
board.display_matrix()
board.display_color()
board.display_occupied()

(1, 1) (2, 1) (3, 1) (4, 1) (5, 1) (6, 1) (7, 1) (8, 1) 
(1, 2) (2, 2) (3, 2) (4, 2) (5, 2) (6, 2) (7, 2) (8, 2) 
(1, 3) (2, 3) (3, 3) (4, 3) (5, 3) (6, 3) (7, 3) (8, 3) 
(1, 4) (2, 4) (3, 4) (4, 4) (5, 4) (6, 4) (7, 4) (8, 4) 
(1, 5) (2, 5) (3, 5) (4, 5) (5, 5) (6, 5) (7, 5) (8, 5) 
(1, 6) (2, 6) (3, 6) (4, 6) (5, 6) (6, 6) (7, 6) (8, 6) 
(1, 7) (2, 7) (3, 7) (4, 7) (5, 7) (6, 7) (7, 7) (8, 7) 
(1, 8) (2, 8) (3, 8) (4, 8) (5, 8) (6, 8) (7, 8) (8, 8) 
B X B B B B B B 
B B B B X B B X 
X X X X X X X B 
X X X X B X X X 
X X X W B X X X 
X W X W X X X W 
W X W X W W W X 
W W W W W W W X 
True False True True True True True True 
True True True True False True True False 
False False False False False False False True 
False False False False True False False False 
False False False True True False False False 
False True False True False False False True 
True False True False True True True False 
True True True True True True True False 


In [203]:
board = Board()
board.current_turn()
board.makeMove("D2","D4")
board.makeMove("E7","E5")
board.makeMove("E2","E4")
board.makeMove("F8","B4") # black bishop check white king

test_move(board.legalMoves("E1"),[(5,7)]) # Test: White King

board.makeMove("D1","D2") # block with queen
board.makeMove("H7","H6") # random moves
board.makeMove("H2","H3") # random moves
board.makeMove("D8","H4") # random moves
test_move(board.legalMoves("D2"),[(2,5),(3,6)]) # Test: White Queen
test_move(board.legalMoves("F2"),[]) # Test: White Pawn pin by Queen
board.display_board()



Test passed
Test passed
Test passed
BR BN BB 41 BK 61 BN BR 
BP BP BP BP 52 BP BP 82 
13 23 33 43 53 63 73 BP 
14 24 34 44 BP 64 74 84 
15 BB 35 WP WP 65 75 BQ 
16 26 36 46 56 66 76 WP 
WP WP WP WQ 57 WP WP 87 
WR WN WB 48 WK WB WN WR 


[ ] - castling
[ ] - en passant
[ ] - is game over?
[ ] - stalemate

In [204]:
# flow of the game

# decide what starting position (startpos or custom fen)


"""
is_game_over? from last turn perspective and this turn perspective
is checkmated?
"""

'\nis_game_over? from last turn perspective and this turn perspective\nis checkmated?\n'

In [205]:
# engine
class Engine:

  def __init__(self,fen):
    self.fen = fen # allow Board class to take fen as input

# input: the game into engine by fen

# output:
  def eval_function(): # output a suggested move
    pass

In [206]:
def to_fen():
  """
  Turn existing board to fen
  """
  pass


#1p5k/1B4P1/1KN4R/nR3Nrb/Q1NrnpP1/7R/3PP3/1P5P w - - 0 1


board = Board(fen="1p5k/1B4P1/1KN4R/nR3Nrb/Q1NrnpP1/7R/3PP3/1P5P w - - 0 1")
board.display_board()
board.to_fen() == "1p5k/1B4P1/1KN4R/nR3Nrb/Q1NrnpP1/7R/3PP3/1P5P w - - 0 1"

11 BP 31 41 51 61 71 BK 
12 WB 32 42 52 62 WP 82 
13 WK WN 43 53 63 73 WR 
BN WR 34 44 54 WN BR BB 
WQ 25 WN BR BN BP WP 85 
16 26 36 46 56 66 76 WR 
17 27 37 WP WP 67 77 87 
18 WP 38 48 58 68 78 WP 


True